In [1]:
import spacy
import pandas as pd
import numpy as np
import tensorflow as tf

from model import DeepmojiNet
from main import prepare_embeddings

tf.config.set_visible_devices([], 'GPU')
nlp = spacy.load('en_core_web_md')

In [2]:
data = pd.read_pickle('../data/kaggle-insults/raw.pickle')
df = pd.DataFrame.from_dict({'info':data['info'],'texts':data['texts']})

In [3]:
df['split'] = -1
df.loc[data['train_ind'],'split'] = 0
df.loc[data['val_ind'],'split'] = 1
df.loc[data['test_ind'],'split'] = 2

In [4]:
# working ot toy example
df = df.sample(4)

In [5]:
#TODO:
# consider applying deepmoji paper filterings
# try different preprocessing schemes: 
# lowercase, spell correct, lemmatization, stemmatization
# remove numerics, stop words, punctiations
# gensim simple_preprocess()

def custom_preprocess(row):
    return [t.text.lower() for t in nlp(row)]


df['cleaned'] = df['texts'].apply(lambda row: custom_preprocess(row))
pd.to_pickle(df,'../data/kaggle-insults/cleaned.pickle')
cleaned_texts = tf.ragged.constant(df.cleaned)

2022-03-27 00:37:55.545939: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
df

,info,texts,split,cleaned
3895,{'label': 0},thank you j .. you are so kind .. enjoy your w...,0,"[thank, you, j, .., you, are, so, kind, .., en..."
2363,{'label': 0},where are you going with this ?,0,"[where, are, you, going, with, this, ?]"
1051,{'label': 0},votto was in our farm before krusty ever came ...,0,"[votto, was, in, our, farm, before, krusty, ev..."
2225,{'label': 1},"it was his idiot parents , idiot .",0,"[it, was, his, idiot, parents, ,, idiot, .]"


In [7]:
lookup_layer,embedding_layer,info = prepare_embeddings(cfg=None)

In [8]:
embedding_layer(lookup_layer(cleaned_texts[3])).shape

TensorShape([8, 300])

In [9]:
model = DeepmojiNet(lookup_layer,embedding_layer,2)
temp_x = cleaned_texts[3]
temp_x = tf.expand_dims(temp_x,0)

In [10]:
output = model(temp_x)

In [11]:
output.shape

TensorShape([1, 2])